# Basic Model Training with Generated Dataset

This notebook demonstrates how to generate a small dataset using the `roler` package, train a simple neural network model, and evaluate its performance.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from roler.model import ModelPrior
from roler.distributions import *
from roler.datasets import Dataset
from roler.simulation import Simulator

## Generating a Small Dataset

We will generate a small dataset using the `Dataset` class from the `roler` package.

In [2]:
prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

simulator = Simulator()
dataset = Dataset(simulator=simulator, prior=prior)
theta, x = dataset.generate_dataset(samples=100)

print("theta.size", theta.shape)
print("x.size", x.shape)

## Defining a Simple Neural Network Model

We will define a simple feedforward neural network for this demonstration.

In [3]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_dim = x.shape[1]
output_dim = theta.shape[1]
model = SimpleNN(input_dim, output_dim)
print(model)

## Training the Model

We will train the model using Mean Squared Error (MSE) loss and the Adam optimizer.

In [4]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, theta)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

## Evaluating Model Performance

We will evaluate the model's performance by comparing the predicted parameters to the true parameters.

In [5]:
model.eval()
with torch.no_grad():
    predictions = model(x)
    mse = criterion(predictions, theta).item()
    print(f'Mean Squared Error: {mse:.4f}')

print("True Parameters:", theta[:5])
print("Predicted Parameters:", predictions[:5])